In [ ]:
!pip install qiskit
!pip install qiskit-aer

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from scipy.optimize import minimize
from math import pi
import numpy as np
from qiskit.circuit.library import ZGate


In [ ]:
num_facilities = 2
num_cities = 4
num_qubits = num_facilities + num_cities*num_facilities
num_ancilla = 1


In [ ]:
open_costs = [1, 1]
connect_costs = [[1, 3, 3, 3],
          [1, 1, 1, 1]]

In [ ]:
# 创建一个量子电路
qc = QuantumCircuit(num_qubits+num_ancilla,num_qubits)


In [ ]:
# 应用哈达玛门到每个量子比特，以创建超级位置态
for qubit in range(num_qubits):
    qc.h(qubit)

In [ ]:
gamma = Parameter('γ')
beta = Parameter('β')

In [ ]:
for i in range(num_facilities):
    qc.rz(open_costs[i] * gamma, i)

for i in range(num_facilities):
    for j in range(num_cities):
        qc.rzz(connect_costs[i][j] * gamma, i, num_facilities + i * num_cities + j)

In [ ]:
penalty = 10
for i in range(num_facilities):
    for j in range(num_cities):
        qc.cz(i, num_facilities + i * num_cities + j)
        qc.rz(penalty * gamma, num_facilities + i * num_cities + j)
        qc.cz(i, num_facilities + i * num_cities + j)

In [ ]:
z_gate = ZGate().control(num_facilities)
unconnected_penalty = 10
ancilla_qubit = num_qubits
for j in range(num_cities):
    controls = [num_facilities + i * num_cities + j for i in range(num_facilities)]
    qc.append(z_gate, controls + [ancilla_qubit])
    qc.rz(unconnected_penalty * gamma, ancilla_qubit)
    qc.append(z_gate, controls + [ancilla_qubit])

In [ ]:
# 示例：混合哈密顿量 H_B (RX门)
for qubit in range(num_qubits):
    qc.rx(2*beta, qubit)

In [ ]:
for qubit in range(num_qubits):
  qc.measure(qubit,qubit)

In [ ]:
qc.draw()

┌───┐  ┌───────┐                                                      »
 q_0: ┤ H ├──┤ Rz(γ) ├───■───────────────■─────────────────■────────────────»
      ├───┤┌─┴───────┴─┐ │               │                 │                »
 q_1: ┤ H ├┤ Rz(1.1*γ) ├─┼───────■───────┼─────────■───────┼─────────■──────»
      ├───┤└───────────┘ │ZZ(γ)  │       │         │       │         │      »
 q_2: ┤ H ├──────────────■───────┼───────┼─────────┼───────┼─────────┼──────»
      ├───┤                      │       │ZZ(3*γ)  │       │         │      »
 q_3: ┤ H ├──────────────────────┼───────■─────────┼───────┼─────────┼──────»
      ├───┤                      │                 │       │ZZ(3*γ)  │      »
 q_4: ┤ H ├──────────────────────┼─────────────────┼───────■─────────┼──────»
      ├───┤                      │                 │                 │      »
 q_5: ┤ H ├──────────────────────┼─────────────────┼─────────────────┼──────»
      ├───┤                      │ZZ(γ)            │                 │      »
 q_6: ┤ H ├──────────────────────■─────────────────┼─────────────────┼──────»
      ├───┤                                        │ZZ(γ)            │      »
 q_7: ┤ H ├────────────────────────────────────────■─────────────────┼──────»
      ├───┤                                                          │ZZ(γ) »
 q_8: ┤ H ├──────────────────────────────────────────────────────────■──────»
      ├───┤                                                                 »
 q_9: ┤ H ├─────────────────────────────────────────────────────────────────»
      └───┘                                                                 »
q_10: ──────────────────────────────────────────────────────────────────────»
                                                                            »
c: 10/══════════════════════════════════════════════════════════════════════»
                                                                            »
«                                                                              »
« q_0: ─■─────────────────■─────────────────■─────■────────────────────■───────»
«       │                 │                 │     │                    │       »
« q_1: ─┼─────────■───────┼──■──────────────┼──■──┼──■─────────────────┼──■────»
«       │         │       │  │ ┌──────────┐ │  │  │  │                 │  │    »
« q_2: ─┼─────────┼───────■──┼─┤ Rz(10*γ) ├─■──┼──┼──┼──■──────────────┼──┼──■─»
«       │         │          │ └──────────┘    │  │  │  │ ┌──────────┐ │  │  │ »
« q_3: ─┼─────────┼──────────┼─────────────────┼──■──┼──┼─┤ Rz(10*γ) ├─■──┼──┼─»
«       │         │          │                 │     │  │ └──────────┘    │  │ »
« q_4: ─┼─────────┼──────────┼─────────────────┼─────┼──┼─────────────────┼──┼─»
«       │ZZ(3*γ)  │          │                 │     │  │                 │  │ »
« q_5: ─■─────────┼──────────┼─────────────────┼─────┼──┼─────────────────┼──┼─»
«                 │          │ ┌──────────┐    │     │  │                 │  │ »
« q_6: ───────────┼──────────■─┤ Rz(10*γ) ├────■─────┼──■─────────────────┼──■─»
«                 │            └──────────┘          │  │ ┌──────────┐    │  │ »
« q_7: ───────────┼──────────────────────────────────■──┼─┤ Rz(10*γ) ├────■──┼─»
«                 │                                     │ └──────────┘       │ »
« q_8: ───────────┼─────────────────────────────────────┼────────────────────┼─»
«                 │ZZ(γ)                                │                    │ »
« q_9: ───────────■─────────────────────────────────────┼────────────────────┼─»
«                                                       │ ┌──────────┐       │ »
«q_10: ─────────────────────────────────────────────────■─┤ Rz(10*γ) ├───────■─»
«                                                         └──────────┘         »
«c: 10/════════════════════════════════════════════════════════════════════════»
«                                                                              »
«                              

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

In [ ]:
def calculate_cost(counts):
    total_cost = 0
    total_counts = sum(counts.values())

    # 计算成本
    for bitstring, count in counts.items():
        # 解析量子比特的状态
        facility_status = bitstring[:num_facilities]  # 前两个比特表示设施状态
        city_connections = bitstring[num_facilities:]  # 接下来的比特表示城市连接
        # 计算开放成本
        for i in range(num_facilities):
            if facility_status[i] == '1':  # 如果设施开放
                total_cost += open_costs[i] * count

        # 计算连接成本
        for i in range(num_facilities):
            for j in range(num_cities):
                if facility_status[i] == '1' and city_connections[i * num_cities + j] == '1':  # 如果城市连接到设施
                    total_cost += connect_costs[i][j] * count
                if facility_status[i] == '0' and city_connections[i * num_cities + j] == '1':  # 如果设施关闭但城市连接到设施
                    total_cost += penalty * count
        res = 0
        for j in range(num_cities):
            for i in range(num_facilities):
                res += int(city_connections[i * num_cities + j])
            if res < 1:
                total_cost += unconnected_penalty * count
            res = 0

    # 归一化成本
    normalized_cost = total_cost / total_counts
    return normalized_cost

In [ ]:
def calculate_gradient(params):
    param_values = {gamma: [params[0]], beta: [params[1]]}

    # 在当前参数下运行量子电路并计算成本
    job = execute(qc, simulator, shots=10000, parameter_binds=[param_values])
    result = job.result()
    counts = result.get_counts(qc)
    cost = calculate_cost(counts)  # 根据 counts 计算成本或分数

    # 计算 gamma 的梯度
    epsilon = 0.01  # 增加的微小偏移量
    param_values_gamma_plus = {gamma: [params[0] + epsilon], beta: [params[1]]}
    job_gamma_plus = execute(qc, simulator, shots=10000, parameter_binds=[param_values_gamma_plus])
    result_gamma_plus = job_gamma_plus.result()
    counts_gamma_plus = result_gamma_plus.get_counts(qc)
    dgamma = (calculate_cost(counts_gamma_plus) - cost) / epsilon

    # 计算 beta 的梯度
    epsilon = 0.01  # 增加的微小偏移量
    param_values_beta_plus = {gamma: [params[0]], beta: [params[1] + epsilon]}
    job_beta_plus = execute(qc, simulator, shots=10000, parameter_binds=[param_values_beta_plus])
    result_beta_plus = job_beta_plus.result()
    counts_beta_plus = result_beta_plus.get_counts(qc)
    dbeta = (calculate_cost(counts_beta_plus) - cost) / epsilon

    return np.array([dgamma, dbeta])

In [ ]:
def objective_function(params):
    # 更新电路参数
    param_values = {gamma: [params[0]], beta: [params[1]]}
    job = execute(qc, simulator, shots=1000, parameter_binds=[param_values])
    result = job.result()
    counts = result.get_counts(qc)

    # 计算目标函数值
    # 这里应该根据您的问题特定计算目标函数
    cost = calculate_cost(counts)  # 根据counts计算成本或分数
    return cost

In [ ]:
params = np.random.rand(2)
print("Initial Parameters:", params)
learning_rate = 0.001  # 学习率
max_iterations = 200  # 最大迭代次数
epsilon = 0.001  # 用于数值梯度计算的微小偏移量

Initial Parameters: [0.90894025 0.51966294]


In [ ]:
for iteration in range(max_iterations):
    gradient = calculate_gradient(params)
    params -= learning_rate * gradient

    # 计算当前参数下的成本
    param_values = {gamma: [params[0]], beta: [params[1]]}
    job = execute(qc, simulator, shots=1000, parameter_binds=[param_values])
    result = job.result()
    counts = result.get_counts(qc)
    cost = calculate_cost(counts)  # 根据 counts 计算成本或分数

    print(f"Iteration {iteration + 1}: Cost = {cost}")

# 输出最优参数和目标函数值
print("Optimal Parameters:", params)
print("Optimal Cost:", cost)

Iteration 1: Cost = 28.77639999999996
Iteration 2: Cost = 28.18659999999998
Iteration 3: Cost = 27.97389999999997
Iteration 4: Cost = 26.24909999999998
Iteration 5: Cost = 26.535899999999998
Iteration 6: Cost = 28.123399999999975
Iteration 7: Cost = 27.642999999999976
Iteration 8: Cost = 26.687799999999964
Iteration 9: Cost = 27.427099999999975
Iteration 10: Cost = 27.336599999999958
Iteration 11: Cost = 26.229599999999984
Iteration 12: Cost = 26.612399999999994
Iteration 13: Cost = 26.251899999999996
Iteration 14: Cost = 26.239399999999993
Iteration 15: Cost = 27.03709999999997
Iteration 16: Cost = 26.092399999999977
Iteration 17: Cost = 26.808099999999975
Iteration 18: Cost = 26.105199999999986
Iteration 19: Cost = 27.02789999999998
Iteration 20: Cost = 26.866399999999988
Iteration 21: Cost = 26.337199999999967
Iteration 22: Cost = 25.363599999999973
Iteration 23: Cost = 26.6985
Iteration 24: Cost = 26.02109999999996
Iteration 25: Cost = 26.991499999999984
Iteration 26: Cost = 27.170

In [ ]:
# 使用最优参数运行电路
optimal_param_values = {gamma: [params[0]], beta: [params[1]]}
job = execute(qc, simulator, shots=1000, parameter_binds=[optimal_param_values])
result = job.result()
optimal_counts = result.get_counts(qc)


In [ ]:
print(calculate_cost(optimal_counts))

26.373999999999985


In [ ]:
# 对测量结果进行排序
sorted_counts = sorted(optimal_counts.items(), key=lambda x: x[1], reverse=True)

# 打印排序后的结果
for bitstring, count in sorted_counts[:10]:
    print(f"Bitstring: {bitstring}, Count: {count}, Cost: {calculate_cost({bitstring:count})}")



Bitstring: 1111000101, Count: 128, Cost: 18.1
Bitstring: 1111000111, Count: 77, Cost: 9.100000000000001
Bitstring: 1111000000, Count: 64, Cost: 26.1
Bitstring: 1110000111, Count: 33, Cost: 6.1000000000000005
Bitstring: 0111000111, Count: 32, Cost: 24.1
Bitstring: 1011000111, Count: 29, Cost: 35.0
Bitstring: 1101000111, Count: 27, Cost: 18.099999999999998
Bitstring: 1111000010, Count: 21, Cost: 17.1
Bitstring: 1001000101, Count: 21, Cost: 44.0
Bitstring: 1110000010, Count: 20, Cost: 24.1


In [ ]:
def validate_condition(count):
  bitstring = count[0]
  facility_status = bitstring[:num_facilities]  # 前两个比特表示设施状态
  city_connections = bitstring[num_facilities:]
  if facility_status.count('1') == 0:
    return False
  for i in range(num_facilities):
      for j in range(num_cities):
        if facility_status[i] == '0' and city_connections[i * num_cities + j] == '1':
          return False
  for j in range(num_cities):
    res = 0
    for i in range(num_facilities):
      if facility_status[i] == '1':
        res += int(city_connections[i * num_cities + j])
    if res < 1:
      return False
  return True

In [ ]:
def finalize_condition(bitstring):
  facility_status = bitstring[:num_facilities]  # 前两个比特表示设施状态
  city_connections = bitstring[num_facilities:]
  res = ['0' for i in range(num_facilities+num_facilities*num_cities)]
  for i in range(num_facilities):
    if facility_status[i] == '1':
      res[i] = '1'
      for j in range(num_cities):
        if city_connections[i * num_cities + j] == '1':
          res[num_facilities + j] = '1'
  for j in range(num_cities):
    min = -1
    min_index = 0
    for i in range(num_facilities):
      if city_connections[i * num_cities + j] == '1':
        if connect_costs[i][j] < min or min == -1:
          min = connect_costs[i][j]
          min_index = i
    for i in range(num_facilities):
      if i == min_index:
        res[num_facilities+i * num_cities + j] = '1'
      else:
        res[num_facilities+i * num_cities + j] = '0'

  return ''.join(res)


In [ ]:
validated = filter(validate_condition, sorted_counts)
validated_list = list(validated)
print(validated_list)

[('1111000111', 77), ('1110000111', 33), ('1111100101', 11), ('1111100111', 5), ('1111010111', 4), ('1110001111', 3), ('1111001111', 3), ('1111110000', 2), ('1110010111', 2), ('1111010010', 2), ('1111100001', 1), ('1111000011', 1)]


In [ ]:
for i in validated_list:
  final = finalize_condition(i[0])
  print(f"Bitstring: {final}, Count: {i[1]}, Cost: {calculate_cost({final:i[1]})}")

Bitstring: 1110000111, Count: 77, Cost: 6.1
Bitstring: 1110000111, Count: 33, Cost: 6.1000000000000005
Bitstring: 1110100101, Count: 11, Cost: 8.1
Bitstring: 1110000111, Count: 5, Cost: 6.1
Bitstring: 1110000111, Count: 4, Cost: 6.1
Bitstring: 1110000111, Count: 3, Cost: 6.1000000000000005
Bitstring: 1110000111, Count: 3, Cost: 6.1000000000000005
Bitstring: 1111110000, Count: 2, Cost: 12.1
Bitstring: 1110000111, Count: 2, Cost: 6.1
Bitstring: 1111010010, Count: 2, Cost: 10.1
Bitstring: 1111100001, Count: 1, Cost: 10.1
Bitstring: 1111000011, Count: 1, Cost: 8.1


In [ ]:
print(validate_condition(("0100001111",1000)))

True
